In [1]:
import os
import h5py

In [2]:
root_dir = os.getcwd()
pre_t_filepath = os.path.join(root_dir, "mask_rcnn_prostate_pretrained.h5")
resnet_filepath = os.path.join(root_dir, "logs/prostate20180227T2229/mask_rcnn_prostate_0040.h5")

In [15]:
def combine_tumor_classifier(pre_t_filepath, resnet_filepath):
    """This function combine the pretrained maskrcnn weights with the tumor classifier weights.
    Function copy the tumor head from resnet file to pretrained file and save in the pretrained
    file.
    # Arguments
    pre_t_filepath: specify the file location of the pretrained maskrcnn
    resnet_filepath: specify the file location of the trained tumor classifier head
    """
    
    # read in the file
    pre_t_f = h5py.File(pre_t_filepath, mode='r+')
    resnet_f = h5py.File(resnet_filepath, mode='r')
    
    # delete all the tumor weights in the pre_t_f
    for name in pre_t_f:
        if name.startswith('tumor'):
            del pre_t_f[name]
    
    # copy the tumor head from resnet_f to pre_t_f
    for name in resnet_f:
        if name.startswith('tumor'):
            resnet_f.copy('tumor_class_conv1', pre_t_f)
    
    pre_t_f.close()
    resnet_f.close()

In [16]:
combine_tumor_classifier(pre_t_filepath, resnet_filepath)

ValueError: Destination object already exists (Destination object already exists)

In [ ]:
# method to fetch the numpy data array from the h5 dataset object
# weight_pre = kenel_pre[()]